# Python Requests File Move

Create, Upload, Download CSV file in the Default workspace only

## Imports

Import necessary python modules.

In [1]:
import os
import requests
import urllib3
from datetime import datetime
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Define Web Service Command Syntax for Requests

In [2]:
class WebServices(object):

    def __init__(self, host_url="", session=None):
        self.host_url = host_url
        self.session = session if session != None else requests.Session()

    def set_api_key(self, api_key):
        self.headers = {'x-ni-api-key':api_key}
        self.auth = ""
        
    def set_usr_pwd(self, username, password):
        self.headers = ""
        self.auth = (username, password)
    
    def post_route(self, url, post_json="", post_files=""):
        print("POST " + self.host_url + url)
        response = self.session.post(self.host_url+url, json=post_json, files=post_files, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response
    
    def get_route(self, url):
        print("GET " + self.host_url + url)
        response = self.session.get(self.host_url+url, json={}, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

    def put_route(self, url, put_json):
        print("PUT " + self.host_url + url)
        response = self.session.put(self.host_url+url, json=put_json, verify=False, headers=self.headers, auth=self.auth)
        print(str(response))
        return response

## File I/O Functions

In [3]:
def make_local_file(file_name, file_content):
    file_path = file_name # file paths are by default relative to the running notebook file in JupyterHub
    f = open(file_path, "w", encoding="utf-8")
    f.write(file_content)
    f.close()
    print(str('Created "' + file_name + '" ==> ' + file_content))
    return file_path

In [4]:
def upload_file(webServices, file_path, file_name):
    f = open(file_path, 'r')
    file_content = f.read()
    f.close()
    files = {'file': (file_name, file_content)}
    response = webServices.post_route("nifile/v1/service-groups/Default/upload-files", post_files=files)
    response_json = response.json()
    print(str(response_json))
    file_id = str(response_json['uri'])[-24:]
    return file_id

In [5]:
def download_file(webServices, file_id):
    response = webServices.get_route("nifile/v1/service-groups/Default/files/" + file_id + "/data?inline=false")
    response_body = response.content
    file_body = str(response_body.decode()).replace("\r", "")
    return file_body

In [6]:
def file_io(webServices): # assumes you are ONLY using the Default workspace
    file_name = "Example File.csv"
    now = datetime.today().isoformat() # "2019-09-05T18:45:08Z"
    file_path = make_local_file(file_name, str(now))
    print(file_path)
    file_id = upload_file(webServices, file_path, file_name)
    print(file_id)
    file = download_file(webServices, file_id)
    print(str(file))
    return file_id

## Connect to SystemLink, Run File I/O, Close Connection

In [7]:
# Connection Info
host_url = "https://localhost:443/"
host_url = os.getenv("SYSTEMLINK_HTTP_URI") # available only in JupyterHub on SystemLink server
api_key = os.getenv("SYSTEMLINK_API_KEY")   # available only in JupyterHub on SystemLink server
print(str(api_key))

# Return WebServices object (set host_url)
webServices = WebServices(host_url)

# Define the connection credentials using an ApiKey or username/password
webServices.set_api_key(api_key)
#webServices.set_usr_pwd("admin", "mypassword")

# perform tag operations
file_io(webServices)
print("done")

wSo_Orl4sRY5a0n922pDjsFp6kPlKFhl2kdFFQQ5Gz
Created "Example File.csv" ==> 2023-05-10T16:30:28.927747
Example File.csv
POST https://bradt2-lt/nifile/v1/service-groups/Default/upload-files
<Response [201]>
{'uri': '/nifile/v1/service-groups/Default/files/645c0cf5e49e4dd11462c2cd'}
645c0cf5e49e4dd11462c2cd
GET https://bradt2-lt/nifile/v1/service-groups/Default/files/645c0cf5e49e4dd11462c2cd/data?inline=false
<Response [200]>
2023-05-10T16:30:28.927747
done
